## TOC:
* [First Bullet Header](#first-bullet)
* [Second Bullet Header](#second-bullet)

In [77]:
import pandas as pd
import os
import numpy as np

In [453]:
dirPath= "C:\\Users\\John\\Documents\\GitHub\\DataExploration\\KaggleOnlineRetail"
dfRaw = pd.read_csv ( os.path.join ( dirPath, 'online_retail_II.csv') ) #, names =[])
dfRaw.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [447]:
# Read in my country to continent lookup table
dfCountries = pd.read_csv ( os.path.join ( dirPath ,'lkpCountries.csv') ) #, names =[])
dfCountries = dfCountries.rename ( {'CountryName':'Country'}, axis=1, errors='raise')

# Data Cleaning Step

In [ ]:
# This was used in creation of lkpCountries.csv
#countriesSeries = dfClean[ 'Country' ].value_counts()
#dfCountries = pd.DataFrame ({'CountryName':countriesSeries.index})
#dfCountries.to_csv ( os.path.join ( dirPath, 'lkpCountries.csv'), index=False )

In [454]:
dfClean = dfRaw.copy()
dfClean ['SalesAmount'] = dfClean['Quantity'] * dfClean['Price']

In [81]:
# Customer ID

# Missing Values - there are none
#dfClean [dfClean ['Customer ID'] == np.NaN] ## None

In [87]:
# Country
#dfClean [ dfClean ['Country'] == '' ] 
#dfClean [ dfClean ['Country'] == ' ' ] 

#### Flag December 2011 Transactions as Censored / Remove 2011 Trans

In [455]:
boolIndex = (dfClean ['InvoiceDate'] >= '2009-12-01') & (dfClean ['InvoiceDate'] <= '2009-12-31')
dfClean['Keep'] = 'Yes'
dfClean.loc[boolIndex,'Keep'] = 'No'
# Added 12/12/2021, easier to remove 2009 transactions
dfClean = dfClean[dfClean['Keep']=='Yes']

# Create Fact Transactions

In [456]:
dfTransactions = dfClean[['Quantity', 'InvoiceDate','Invoice' , 'SalesAmount', 'StockCode', 'Customer ID', 'Country']]
dfTransactions.head()

,Quantity,InvoiceDate,Invoice,SalesAmount,StockCode,Customer ID,Country
45228,5,2010-01-04 09:24:00,493410,22.50,TEST001,12346.0,United Kingdom
45229,-1,2010-01-04 09:43:00,C493411,-4.25,21539,14590.0,United Kingdom
45230,5,2010-01-04 09:53:00,493412,22.50,TEST001,12346.0,United Kingdom
45231,1,2010-01-04 09:54:00,493413,0.85,21724,NaN,United Kingdom
45232,1,2010-01-04 09:54:00,493413,3.75,84578,NaN,United Kingdom


### Specify year one and year two

In [458]:
yr1Bool = (dfClean['InvoiceDate'] >= '2009-12-01') & (dfClean['InvoiceDate'] < '2010-12-01')
yr2Bool = (dfClean['InvoiceDate'] >= '2010-12-01') & (dfClean['InvoiceDate'] < '2011-12-01')

dfTransactions['Analysis_Year'] = 'Other'
dfTransactions.loc [yr1Bool, 'Analysis_Year'] = 'Yr1'
dfTransactions.loc [yr2Bool, 'Analysis_Year'] = 'Yr2'

# Create Customer Dimension

In [459]:
dfCustomers = dfClean.groupby('Customer ID').agg({'Quantity': 'sum', 'SalesAmount': 'sum'}).reset_index()

#dfCustomers.head()

In [460]:
dfCustomers['SalesAmountDecile'] = 10 - pd.qcut(dfCustomers['SalesAmount'], 10,  labels = False) 


In [461]:
# Bin the SalesAmount column
bins = [600000,300000,200000,100000,50000,20000,10000,5000,1000,500,0,-100000000]
bins.reverse()
labels = ['300,001-600,000', '200,001-300,000', '100,001-200,000','50,001-100,000','20,000-50,000','10,000-20,000','5,000-10,000','1,001-5,000','501-1000','0-500','Under Zero'   ]
labels.reverse()
dfCustomers['SalesAmountBanded'] =pd.cut( dfCustomers['SalesAmount'] , bins,  labels=labels)

# Check the bandings
dfCustomers.groupby('SalesAmountBanded').agg({'SalesAmount': ['min','max']}  ).reset_index()

#dfCustomers.head()

SalesAmountBanded   SalesAmount           
                               min        max
0         Under Zero -2.511109e+04       0.00
1              0-500  1.776357e-15     499.98
2           501-1000  5.002400e+02     999.70
3        1,001-5,000  1.000160e+03    4991.61
4       5,000-10,000  5.005460e+03    9982.31
5      10,000-20,000  1.000691e+04   19787.13
6      20,000-50,000  2.006177e+04   48975.65
7     50,001-100,000  5.154792e+04   96487.49
8    100,001-200,000  1.042197e+05  170397.61
9    200,001-300,000  2.335794e+05  285539.97
10   300,001-600,000  5.086456e+05  557209.48

### Calculate Number of Invoices Over Period (Orders)

In [463]:
dfTransactionsSummary = dfTransactions.groupby ('Customer ID')['Invoice'].nunique().reset_index()
dfTransactionsSummary = dfTransactionsSummary.rename ({'Invoice':'Number_Invoices'}, axis=1)
dfCustomers = dfCustomers.merge (dfTransactionsSummary, on =['Customer ID'], how='left')

### Calculate Number of InvoiceDates Over the Period

In [464]:
dfTransactionsSummary = dfTransactions.groupby ('Customer ID')['InvoiceDate'].nunique().reset_index()
dfTransactionsSummary = dfTransactionsSummary.rename ({'InvoiceDate':'Number_Invoices Dates'}, axis=1)
dfCustomers = dfCustomers.merge (dfTransactionsSummary, on =['Customer ID'], how='left')


### Calculate First Last Transactions Dates in Each Year and the Frequency in each Year and Overall

In [465]:
# First, Last Transactions in Each Year for each CustomerID
dfTransactionDateSummaries = dfTransactions.groupby (['Customer ID']).agg({'InvoiceDate': ['min','max']  }).reset_index()
lvl0 = dfTransactionDateSummaries.columns.get_level_values(0)
lvl1 = dfTransactionDateSummaries.columns.get_level_values(1)
dfTransactionDateSummaries.columns = lvl0+lvl1

#dfTransactions = dfTransactions.merge (dfTransactionDateSummaries, on=['Customer ID', 'Analysis_Year'] )

dfCustomers = dfCustomers.merge (dfTransactionDateSummaries, on =['Customer ID'] )
dfCustomers.head()

,Customer ID,Quantity,SalesAmount,SalesAmountDecile,SalesAmountBanded,Number_Invoices,Number_Invoices Dates,InvoiceDatemin,InvoiceDatemax
0,12346.0,26,-178.18,10,Under Zero,12,12,2010-01-04 09:24:00,2011-01-18 10:17:00
1,12347.0,3286,5633.32,1,"5,000-10,000",8,8,2010-10-31 14:20:00,2011-12-07 15:52:00
2,12348.0,2714,2019.40,3,"1,001-5,000",5,5,2010-09-27 14:59:00,2011-09-25 13:13:00
3,12349.0,1624,4428.69,2,"1,001-5,000",4,4,2010-04-29 13:20:00,2011-11-21 09:51:00
4,12350.0,197,334.40,8,0-500,1,1,2011-02-02 16:01:00,2011-02-02 16:01:00


# DIM Countries

In [466]:
dfCountryBandings =  dfClean.groupby('Country').agg({'Quantity': 'sum', 'SalesAmount': 'sum'}).reset_index()
# Deciles
dfCountryBandings['CountrySalesAmountQuintile'] =  pd.qcut(dfCountryBandings['SalesAmount'], 5,  labels = False) + 1
# Ranking
dfCountryBandings['CountryRankSalesAmount'] = dfCountryBandings['SalesAmount'].rank()
#dfCountryBandings.sort_values ('SalesAmount', ascending=False)

In [467]:
# Join the two country data frames together
#dfCountryBandings['Country']
dfCountries['Country']
dfCountries= dfCountries.merge (dfCountryBandings, on='Country' , how='left')
dfCountries.head()

,Country,Continent,SubContinent,Quantity,SalesAmount,CountrySalesAmountQuintile,CountryRankSalesAmount
0,United Kingdom,Europe,UK / Ireland,8310667,1.565493e+07,5,43.0
1,EIRE,Europe,UK / Ireland,321979,5.963916e+05,5,42.0
2,Germany,Europe,N. Europe,219880,4.082059e+05,5,40.0
3,France,Europe,N. Europe,181700,3.226368e+05,5,39.0
4,Netherlands,Europe,N. Europe,371129,5.333202e+05,5,41.0


# Output CSVs

In [470]:
dfCustomers.to_csv ( os.path.join ( dirPath, 'Outputs', 'DIM_Customers.csv'), index=False )
dfTransactions.to_csv ( os.path.join ( dirPath,'Outputs', 'FACT_Transactions.csv'), index=False )
dfCountries.to_csv ( os.path.join ( dirPath,'Outputs', 'DIM_Countries.csv'), index=False )

In [181]:

#dfTmp = dfCustomers.groupby('SalesAmountDecile').agg({'SalesAmount': 'sum'}).reset_index()
#dfTmp.to_csv ( os.path.join ( dirPath, 'Outputs', 'dfTmp.csv'), index=False )
